# T-SQL Scripts to create database objects leveraging vector

First, we will create stored procedures that can take a pre-generated embedding and update the appropriate row (identified by @id) with the provided embedding.  We do this for both speakers and sessions.  You will need to install the polyglot notebook extension in VS Code to run this.

In [1]:
#r "nuget:Microsoft.DotNet.Interactive.SqlServer,*-*"


Installed Packages Microsoft.DotNet.Interactive.SqlServer, 1.0.0-beta.24563.1

Loading extension script from `C:\Users\katsmith\.nuget\packages\microsoft.dotnet.interactive.sqlserver\1.0.0-beta.24563.1\interactive-extensions\dotnet\extension.dib`

Query Microsoft SQL Server databases. 
 This extension adds support for connecting to Microsoft SQL Server databases using the #!connect mssql magic command. For more information, run a cell using the #!sql magic command.

In [2]:
#!connect mssql --kernel-name SessionsDB "Data Source=x6eps4xrq2xudenlfv6naeo3i4-f5kss76426duzhm5wobzrpzfmi.msit-database.fabric.microsoft.com,1433;Initial Catalog=Sessions_database-78649ca8-cab7-4552-b099-28ade81e7832;Multiple Active Result Sets=False;Connect Timeout=30;Encrypt=True;Trust Server Certificate=False;Authentication=Active Directory Interactive"

Kernel added: #!sql-SessionsDB

In [3]:
-- Simply updates the session embedding with the embedding passed in.
create or alter procedure update_session_embeddings
@id int,
/** 
    Due to a bug, we are currently using Varchar(MAX).
    Once the bug is fixed, use Vector(1536) like you would expect.
**/  
@in_embeddings Varchar(MAX) /** Vector(1536) **/
as

update 
    sessions 
set 

    embeddings = @in_embeddings,
    require_embeddings_update = 0
where   
    id = @id

GO

Commands completed successfully.

In [4]:
-- Simply updates speaker embedding with embedding passed in
create or alter procedure update_speaker_embeddings
@id int,
/** 
    Due to a bug, we are currently using Varchar(MAX).
    Once the bug is fixed, use Vector(1536) like you would expect.
**/  
@in_embeddings Varchar(MAX) /** Vector(1536) **/
as

update 
   speakers 
set 
    embeddings = @in_embeddings,
    require_embeddings_update = 0
where   
    id = @id
    
GO

Commands completed successfully.

Now we create simple stored procedures that will look through the entire sessions and speakers tables, respectively - for every entry that requires an embeddings update, we will generate the embedding and update the row.  Note: this code is written for clarity, not necessarily for production purposes.

In [5]:
-- Update all sessions

create or alter procedure update_all_sessions

as
begin

set nocount on;

declare @id int, @abstract NVARCHAR(max),  @message NVARCHAR(max)

declare session_cursor cursor FAST_FORWARD READ_ONLY
for select id, abstract from sessions 
where require_embeddings_update = 1
order by id

open session_cursor

fetch next from session_cursor
INTO @id, @abstract

while @@FETCH_STATUS = 0
begin

  declare @retval int, @embedding Vector(1536);
  -- Call the get_embedding stored procedure to retrieve an abstract embedding
  exec @retval = get_embedding @abstract, @embedding OUTPUT

  -- Call the update_session_embeddings stored procedure to update the session embeddings
  exec @retval = update_session_embeddings @id, @embedding

fetch next from session_cursor
INTO @id, @abstract

end
close session_cursor
deallocate session_cursor

end
GO

Commands completed successfully.

In [6]:
-- Update all speakers

create or alter procedure update_all_speakers

as
begin

set nocount on;

declare @id int, @full_name NVARCHAR(100),  @message NVARCHAR(max)

declare speaker_cursor cursor FAST_FORWARD READ_ONLY
for select id, full_name from speakers 
where require_embeddings_update = 1
order by id

open speaker_cursor

fetch next from speaker_cursor
INTO @id, @full_name

while @@FETCH_STATUS = 0
begin

  declare @retval int, @embedding Vector(1536);
  -- Call the get_embedding stored procedure to retrieve an abstract embedding
  exec @retval = get_embedding @full_name, @embedding OUTPUT

  -- Call the update_session_embeddings stored procedure to update the session embeddings
  exec @retval = update_speaker_embeddings @id, @embedding

fetch next from speaker_cursor
INTO @id, @full_name

end
close speaker_cursor
deallocate speaker_cursor

end
go

Commands completed successfully.

Select from these tables to see that embeddings need to be generated

In [7]:
select title, CAST(embeddings as  VARCHAR(MAX)) as embeddings_vector
from Sessions
where require_embeddings_update = 1

select full_name, CAST(embeddings as  VARCHAR(MAX)) as embeddings_vector
from speakers
where require_embeddings_update = 1

(4 rows affected)

(4 rows affected)

title,embeddings_vector
Reducing Plastic Waste: Community-Led Initiatives,<null>
Robotics for Everyday Life: Innovations and Applications,<null>
Medieval European History: Uncovering the Past,<null>
Innovations in Cancer Treatment: A Biomedical Researcher's Perspective,<null>


full_name,embeddings_vector
Amina El-Sayed,<null>
Hiroshi Tanaka,<null>
Liam O'Connor,<null>
Priya Patel,<null>


Now, we can call these two stored procedures to demonstrate that they do update the sessions and speakers already entered - this should update the current data appropriately.

In [8]:
-- Update existing data with embeddings
exec update_all_sessions

exec update_all_speakers

Commands completed successfully.

Select from the tables again - we can see that the embeddings have been generated

In [9]:
select title, substring(cast(embeddings as  VARCHAR(MAX)), 1, 10) as embeddings_string_prefix
from Sessions

select full_name, substring(cast(embeddings as  VARCHAR(MAX)), 1, 10) as embeddings_string_prefix
from Speakers

(4 rows affected)

(4 rows affected)

title,embeddings_string_prefix
Reducing Plastic Waste: Community-Led Initiatives,[1.3763197
Robotics for Everyday Life: Innovations and Applications,[-1.545385
Medieval European History: Uncovering the Past,[9.1628600
Innovations in Cancer Treatment: A Biomedical Researcher's Perspective,[6.3478379


full_name,embeddings_string_prefix
Amina El-Sayed,[-2.515922
Hiroshi Tanaka,[-8.858545
Liam O'Connor,[-4.204304
Priya Patel,[1.1320229


To keep things updated as we move forward - we need to create triggers that will ensure new data has up-to-date embeddings.  
Note that instead of calling the procedures just created, we define the trigger to operate over the inserted set of rows only.

In [11]:
/**
Create a trigger for sessions
**/

create or alter trigger GetSessionEmbeddingsOnInsert
on sessions
after insert
as

declare session_cursor cursor FAST_FORWARD READ_ONLY


-- Note here that we are just operating over the inserted rows
for select id, abstract from inserted
where require_embeddings_update = 1
order by id


set nocount on;

declare @id int, @abstract NVARCHAR(max),  @message NVARCHAR(max)

open session_cursor

fetch next from session_cursor
INTO @id, @abstract

while @@FETCH_STATUS = 0
begin

   declare @retval int, @embedding Vector(1536);
  -- Call the get_embedding stored procedure to retrieve an abstract embedding
  exec @retval = get_embedding @abstract, @embedding OUTPUT

  -- Call the update_session_embeddings stored procedure to update the session embeddings
  exec @retval = update_session_embeddings @id, @embedding

  fetch next from session_cursor
INTO @id, @abstract

end
close session_cursor
deallocate session_cursor



Commands completed successfully.

In [12]:
/**  
Create a trigger for speakers
**/
create or alter trigger GetSpeakerEmbeddingsOnInsert
on speakers
after insert
as

declare speaker_cursor cursor FAST_FORWARD READ_ONLY

-- Notice here that we are just operating over inserted rows.
for select id, full_name from inserted
where require_embeddings_update = 1
order by id

set nocount on;

declare @id int, @full_name NVARCHAR(100),  @message NVARCHAR(max)

open speaker_cursor

fetch next from speaker_cursor
INTO @id, @full_name

while @@FETCH_STATUS = 0
begin

   declare @retval int, @embedding Vector(1536);
  -- Call the get_embedding stored procedure to retrieve an abstract embedding
  exec @retval = get_embedding @full_name, @embedding OUTPUT

  -- Call the update_speaker_embeddings stored procedure to update the session embeddings
  exec @retval = update_speaker_embeddings @id, @embedding

  fetch next from speaker_cursor
INTO @id, @full_name

end
close speaker_cursor
deallocate speaker_cursor


Commands completed successfully.